Detección de caras con webcam

In [2]:
import cv2
import time
import FaceNormalizationUtils as faceutils
# My face detectors interface
import FaceDetectors

En el bucle de procesamiento, las teclas 'd' y 'e' `permiten respectivamenet cambiar de modelo de detección de caras, y en su caso de máscara de detección del rostro.

La ejecución de la siguiente celda produce error al no disponer de los archivos shape_predictor_5_face_landmarks.dat y shape_predictor_68_face_landmarks.dat que por su tamaño no se incluyeron en el repositorio. Pueden descargarse desde el enlace proporcionado en el campus virtual (opción aconsejada), o
desde el [repositorio de archivos de dlib](http://dlib.net/files/).



In [3]:
normalizatorHS = faceutils.Normalization()

# Face detectors interface
FDet = FaceDetectors.FaceDetector()

# Fonts
font = cv2.FONT_HERSHEY_SIMPLEX

# Webcam connection
cap = cv2.VideoCapture(0)
# Check for other cameras
if not cap.isOpened():
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            print('Camera error')
            exit(0)
        else:
            print('Camera 0')
    else:
        print('Camera 1')
else:
    print('Camera 0')

    # Face detection and eye model setup
imodoF = 0
imodoE = 0

debug = 0

#Set camera resolution
cap.set(3,640);
cap.set(4,480);

while True:
    # Get frame
    t = time.time()
    ret, frame = cap.read()
    # For HS normalization
    B, G, R = cv2.split(frame)

    # Search face with a specific setup for face and eye detection
    values = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[imodoF], FDet.EyeDetectors[imodoE])
    if values is not None:
        face, eyes, shape = values

        #draws face container
        [x, y , w, h] = face
        if x > -1:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

            # draws eyes and mask if available
            [lex, ley, rex, rey] = eyes
            if lex > -1:
                # Show detected facial elements
                if imodoF > 0:
                    for (x, y) in shape:
                        cv2.circle(frame, (x, y), 2, (255, 255, 255), -1)

                cv2.circle(frame, ((int)(lex), (int)(ley)), 4, (0, 0, 255), -1)
                cv2.circle(frame, ((int)(rex), (int)(rey)), 4, (0, 255, 0), -1)


                # Normalize and show
                # color channels
                normalizatorHS.normalize_gray_img(B, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Bnorm = normalizatorHS.normf_image
                normalizatorHS.normalize_gray_img(G, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Gnorm = normalizatorHS.normf_image
                normalizatorHS.normalize_gray_img(R, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Rnorm = normalizatorHS.normf_image
                NormBGR = cv2.merge((Bnorm, Gnorm, Rnorm))
                cv2.imshow("Normalized", NormBGR)


    if debug:
        print("Processing time : {:.3f}".format(time.time() - t))

    # Show resulting image
    cv2.putText(frame, FDet.FaceDetectors[imodoF], (10, 20), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    if imodoF == 1 or imodoF == 2:
        cv2.putText(frame, FDet.EyeDetectors[imodoE], (50, 20), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.imshow('Cam', frame)
    
    # Esc to finish
    tec = cv2.waitKey(40)
    if tec & tec == 27:  # Esc
        break
    # Face detector change
    elif tec & 0xFF == ord('d'):
        imodoF = imodoF + 1
        if imodoF >= len(FDet.FaceDetectors):
            imodoF = 0
    #Eye detector change
    elif tec & 0xFF == ord('e'):
        imodoE = imodoE + 1
        if imodoE >= len(FDet.EyeDetectors):
            imodoE = 0

# Close windoews and release camera
cap.release()
cv2.destroyAllWindows()

Camera 0


In [6]:
import cv2
import numpy as np
import dlib
from math import hypot
from math import atan2
from math import pi
import argparse

def rotate_image(image, angle, filter_name):
    image_center = (775,700)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return result


filter_name = 'thug.png'
showLandmark = 'true'


cap = cv2.VideoCapture(0)


detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")


while True:
    Sucess, img = cap.read()
    img = cv2.flip(img, 1)
    
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = detector(img)
    for face in faces:
        landmarks = predictor(imgGray, face)
        top_nose = (landmarks.part(29).x, landmarks.part(29).y)
        center_nose = (landmarks.part(30).x, landmarks.part(30).y)
        left_nose = (landmarks.part(31).x, landmarks.part(31).y)
        right_eye = (landmarks.part(45).x, landmarks.part(45).y)
        left_mouth = (landmarks.part(48).x, landmarks.part(48).y)
        right_mouth = (landmarks.part(54).x, landmarks.part(54).y)
        left_eye = (landmarks.part(36).x, landmarks.part(36).y)
        right_nose = (landmarks.part(35).x, landmarks.part(35).y)
        up_mouth = (landmarks.part(51).x, landmarks.part(51).y)
        down_mouth = (landmarks.part(57).x, landmarks.part(57).y)
        down_face = (landmarks.part(8).x, landmarks.part(8).y)
      
        
        front_filter = cv2.imread('./filters/'+filter_name)
        front_filter = cv2.copyMakeBorder(front_filter, 500, 500, 500, 500, cv2.BORDER_CONSTANT, value=[0,0,0,0])
        rotation = -atan2(right_eye[1]-left_eye[1],right_eye[0]-left_eye[0])/pi*180
        
        
        nose_width = int(hypot(left_eye[0]-right_eye[0], left_eye[1]-right_eye[1])*4.6)
        nose_height = int(nose_width * 0.6708*1)
        offset = int(nose_width*18/200)
        top_left = (int(center_nose[0]-nose_width/2),int(center_nose[1]-nose_height/2)-offset)
        bottom_right = (int(center_nose[0]+nose_width/2),int(center_nose[1]+nose_height/2)-offset)
        
        ffilter = cv2.resize(front_filter, (nose_width, nose_height))
        filter_gray = cv2.cvtColor(ffilter, cv2.COLOR_BGR2GRAY)
        _, filter_mask = cv2.threshold(filter_gray, 0, 255, cv2.THRESH_BINARY_INV)
        
        filter_area = img[max(0,top_left[1]):min(480,top_left[1]+nose_height),max(0,top_left[0]):min(640,top_left[0]+nose_width)]
        filter_mask = filter_mask[max(0,-top_left[1]):max(0,-top_left[1])+filter_area.shape[0],max(0,-top_left[0]):max(0,-top_left[0])+filter_area.shape[1]]
        
        filter_alpha = cv2.bitwise_and(filter_area, filter_area, mask=filter_mask)
        final_nose = cv2.add(filter_alpha, ffilter[max(0,-top_left[1]):filter_area.shape[0]+max(0,-top_left[1]),max(0,-top_left[0]):max(0,-top_left[0])+filter_area.shape[1]])

        img[max(0,top_left[1]):min(480,top_left[1]+nose_height),max(0,top_left[0]):min(640,top_left[0]+nose_width)] = final_nose
    
    if showLandmark == 'true':
        for i in range(68):
            cv2.circle(img, (landmarks.part(i).x, landmarks.part(i).y), radius=0, color=(0, 0, 255), thickness=5)
        
    cv2.imshow("Filtro", img)
    
    tec = cv2.waitKey(40)
    if tec & tec == 27:  # Esc
        break
    
        
cap.release()
cv2.destroyAllWindows()